In [76]:
import os
import json
import openai
import tiktoken
import numpy as np
from pathlib import Path
from tqdm.auto import tqdm
from datetime import datetime
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)
from dotenv import find_dotenv, load_dotenv

from datasets import load_dataset, Dataset

In [2]:
load_dotenv(find_dotenv())

False

In [3]:
model_id = "gpt-3.5-turbo-1106"
client = openai.OpenAI(api_key=os.environ["OPENAI_API_KEY"])
tokenizer = tiktoken.encoding_for_model(model_id)

In [126]:
prompt_json = """
# TASK DESCRIPTION
Create a dataset for LLM fine-tuning from user queries about industrial robotic operations by formatting them to corresponding JSON function calls.

# JSON PARAMETER VALUES
- functions: Array of function calls derived from the query, identified using separators like "and", "then", ",", etc.
- function_name: Specific robot function ("move_tcp", "move_joint", "get_joint_values").
- input_name: Name of the function's input parameter, specifying the type (integer, float, string).
- input_value: Value of the input parameter, adhering to the specified type.

# FUNCTIONS EXPLAINED WITH EXAMPLES
- move_tcp: Moves the robot's TCP. 
    Inputs: x, y, z (position; type: float), q1, q2, q3, q4 (quaternion values; type: float), unit (correlates to x, y, z; type: str, ["m", "cm", "mm"]). 
    Example: "Move TCP to coordinates (0.5, 0.3, 0.7) m" translates to x: 0.5, y: 0.3, z: 0.7; unit: m
- move_joint: Rotates/moves a robot joint. 
    Inputs: joint (index number; type: array of integers; note: joints go from 0 to n-1), angle (degrees or radians, type: array of floats), unit (correlates to angle; type: str, ["rad", "deg"]). 
    Example: "Rotate joint 2 by 45 degrees" results in joint: [2], angle: [45], unit: deg.
- get_joint_values: Retrieves robot joint statuses, no input parameters required. 
    Example: "Get the current status of robot joints."

# GUIDANCE
- Focus on verbs and technical terms for function selection.
- Include only the necessary functions directly implied by the query.
- Maintain the sequence of functions as in the query.
- Include only necessary functions as per the query.
- Specify units only when required, omit default values.

# RESPONSE JSON FORMAT
{{
    "query": "<generated_query>",
    "functions": [
        {{
            "function_name": "<name>",
            "inputs": [
                {{"name": "<name>", "value": "<value>", "unit": "<unit>"}},
                {{"name": "<name>", "value": "<value>", "unit": "<unit>"}}
            ]
        }},
        {{
            "function_name": "<name>",
            "inputs": [{{"name": "<name>", "value": "<value>", "unit": "<unit>"}}]
        }}
    ]
}}

USER QUERY: Move robot tcp along x and y for 1000mm
RESPONSE:
{{
    'functions': [{{'function_name': 'move_tcp',
        'inputs': [
            {{'name': 'x', 'value': 1000.0, 'unit': 'mm'}},
            {{'name': 'y', 'value': 1000.0, 'unit': 'mm'}}
            ]
        }}
    ]
}}

USER QUERY: Move robot sixth joint for -30 degrees
RESPONSE:
{{
    'functions': [{{'function_name': 'move_joint',
        'inputs': [
            {{'name': 'joint', 'value': [5], 'unit': None}},
            {{'name': 'angle', 'value': [-30.0], 'unit': 'deg'}}
            ]
        }}
    ]
}}

USER QUERY: Give me robot joints info
RESPONSE:
{{
    'functions': [{{'function_name': 'get_joint_values',
        'inputs': []
        }}
    ]
}}

USER QUERY: Rotate robot base for 45 and move TCP along x axis for 50 milimeters.
RESPONSE:
{{
    'functions': [
        {{'function_name': 'move_joint',
            'inputs': [
                {{'name': 'joint', 'value': [0], 'unit': None}},
                {{'name': 'angle', 'value': [45.0], 'unit': 'deg'}}
            ]
        }},
        {{'function_name': 'move_tcp',
            'inputs': [{{'name': 'x', 'value': 50.0, 'unit': 'mm'}}]
        }}
    ]
}}

USER QUERY: I want you to rotate joint 2 for 30 and joint 7 for 45 degrees then joint 3 for pi/4
RESPONSE:
{{
    'functions': [
        {{'function_name': 'move_joint',
            'inputs': [
                {{'name': 'joint', 'value': [2, 7], 'unit': None}},
                {{'name': 'angle', 'value': [30.0, 45.0], 'unit': 'deg'}}
                ]
        }},
        {{'function_name': 'move_joint',
            'inputs': [
                {{'name': 'joint', 'value': [3], 'unit': None}},
                {{'name': 'angle', 'value': [0.785398], 'unit': 'rad'}}
                ]
        }}
    ]
}}

USER QUERY: {user_query}
RESPONSE:
"""

In [119]:
prompt_tokenized = tokenizer.encode(prompt_json)
len(prompt_tokenized)

1097

In [68]:
prompt_queries = """
# TASK DESCRIPTION {rand_num}
Create a dataset for LLM fine-tuning consisting of user queries about industrial robotic operations and their corresponding JSON function calls. Focus on generating queries with varying complexity levels, from simple to advanced, and provide examples in real-world scenarios.

# FUNCTIONS EXPLAINED WITH EXAMPLES
- move_tcp: Moves the robot's TCP. 
    Inputs: x, y, z (position; type: float), q1, q2, q3, q4 (quaternion values; type: float), unit (correlates to x, y, z; type: str, ["m", "cm", "mm"]). 
    Example: "Move TCP to coordinates (0.5, 0.3, 0.7) m" translates to x: 0.5, y: 0.3, z: 0.7; unit: m
- move_joint: Rotates/moves a robot joint. 
    Inputs: joint (index number; type: array of integers; note: joints go from 0 to n-1), angle (degrees or radians, type: array of floats), unit (correlates to angle; type: str, ["rad", "deg"]). 
    Example: "Rotate joint 2 by 45 degrees" results in joint: [2], angle: [45], unit: deg.
- get_joint_values: Retrieves robot joint statuses, no input parameters required. 
    Example: "Get the current status of robot joints."

# GUIDANCE
- Focus on verbs and technical terms for function selection.
- Ensure queries are deterministic and precise.
- Queries should vary in complexity, from direct instructions to those requiring contextual understanding.
- Real-World Scenarios: Frame queries in practical industrial settings.

# RESPONSE FORMAT
- Return format is JSON.
- Only one key: "queries"
- Values are list with strings that are generated queries.
- Generate {num_examples_per_prompt} unique queries.
- Query examples:
    - Move robot tcp along x and y for 1000mm
    - Move robot sixth joint for -30 degrees
    - Give me robot joints info
    - Rotate robot base for 45 and move TCP along x axis for 50 milimeters.
    - I want you to rotate joint 2 for 30 and joint 7 for 45 degrees then joint 3 for pi/4
    - 
- Output format example:
    {{
        "queries" : [<generated_query>, <generated_query>, ..., <generated_query>]
    }}

"""

In [16]:
prompt_tokenized = tokenizer.encode(prompt_queries)
len(prompt_tokenized)

522

In [136]:
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def generate_data(
    temperature: float, messages: list, frequency_penalty: float = None
) -> dict:
    response = client.chat.completions.create(
        model=model_id,
        temperature=temperature,
        response_format={"type": "json_object"},
        frequency_penalty=frequency_penalty,
        messages=messages,
    )
    return json.loads(response.choices[0].message.content)

In [122]:
def save_data(output_path: Path, queries: list) -> None:
    with open(output_path, "w", encoding="utf-8") as file:
        json.dump(queries, file, indent=4, ensure_ascii=False)

### Generate queries


In [ ]:
current_dt = datetime.now().strftime("%Y-%b-%d_%H-%M-%S")
output_path = Path(f"./generated-queries_{current_dt}.json")

In [106]:
num_examples = 1_000
num_examples_per_prompt = 20
iterations = int(num_examples / num_examples_per_prompt)
check_point = 5

queries = []
for iter in tqdm(range(iterations), total=iterations):
    try:
        messages = [
            {
                "role": "user",
                "content": prompt_queries.format(
                    rand_num=np.random.rand().__round__(5),
                    num_examples_per_prompt=num_examples_per_prompt,
                ),
            },
        ]
        result = generate_data(
            temperature=1,
            frequency_penalty=round(np.random.rand() * 0.4, 4),
            messages=messages,
        )

        queries.extend(
            [query for query in result["queries"] if len(query)]
        )  # Check for empty string
    except json.JSONDecodeError as e:
        print(f"ERROR happend while parsing response: {e}")
    if (iter + 1) % check_point == 0:
        save_data(output_path, queries)

save_data(output_path, queries)
print(f"Number of generated queries: {len(queries)}")

  0%|          | 0/50 [00:00<?, ?it/s]

In [141]:
len(queries)

986

### Generate JSONs


In [ ]:
current_dt = datetime.now().strftime("%Y-%b-%d_%H-%M-%S")
output_path = Path(f"./formatted-queries_{current_dt}.json")

In [140]:
check_point = 5

formatted_queries = []
for index, query in tqdm(enumerate(queries), total=len(queries)):
    try:
        messages = [
            {
                "role": "user",
                "content": prompt_json.format(user_query=query),
            }
        ]
        result = generate_data(
            temperature=0,
            messages=messages,
            frequency_penalty=None,
        )
        formatted_queries.append((query, result))

    except json.JSONDecodeError as e:
        print(f"ERROR happend while parsing response: {e}")

    if (index + 1) % check_point == 0:
        save_data(output_path, formatted_queries)

save_data(output_path, formatted_queries)
print(f"Number of formatted queries: {len(formatted_queries)}")

  0%|          | 0/986 [00:00<?, ?it/s]

Number of formatted queries: 986


### Format data for training


In [13]:
with open(
    "./formatted-queries_2024-Jan-30_23-29-10.json", "r", encoding="utf-8"
) as file:
    formatted_queries = json.loads(file.read())

In [14]:
output_path = Path("./processed_dataset_2024-Jan-30_23-29-10.jsonl")

Real data for robot-agent


In [15]:
dataset = [
    {"user_query": sample[0], "function_calling": str(sample[1])}
    for sample in formatted_queries
]

False data for robot-agent <br>
We will create queries that do not relate to functions that we have defined so the agent ca nlearn to distinguish them


In [2]:
dataset_false = load_dataset("Intel/orca_dpo_pairs")["train"]

Generating train split: 0 examples [00:00, ? examples/s]

We do not want to large sentences


In [50]:
dataset_false = dataset_false.filter(lambda x: len(x["question"]) < 500)

In [51]:
np.random.seed(8)
indexes = np.random.randint(0, len(dataset_false) - 1, int(len(dataset) * 0.2))

In [54]:
json_false = """{{"functions": [{{"function_name": "", "inputs": []}}]}}"""

In [55]:
dataset_false = [
    {"user_query": question, "function_calling": json_false}
    for question in dataset_false[indexes]["question"]
]

In [59]:
from random import sample

In [62]:
merged_dataset = dataset + dataset_false
len(merged_dataset)

1183

In [77]:
dataset = Dataset.from_dict({"data": merged_dataset})
split_dataset = dataset.train_test_split(test_size=0.1)

In [79]:
output_path

PosixPath('processed_dataset_2024-Jan-30_23-29-10.jsonl')

In [80]:
split_dataset.save_to_disk("dataset_2024-Jan-30_23-29-10")

Saving the dataset (0/1 shards):   0%|          | 0/1064 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/119 [00:00<?, ? examples/s]

In [ ]:
with open(output_path, "w", encoding="utf-8") as file:
    for entry in dataset:
        json.dump(entry, file)
        file.write("\n")